# Browsing waveforms with the `WaveformBrowser`

This is a tutorial demonstrating several ways to use the `WaveformBrowser` to examine waveform data. This will consist of multiple examples, increasing in complexity, and will use LEGEND test data from [legend-testdata](https://github.com/legend-exp/legend-testdata). The `WaveformBrowser` [[docs]](https://pygama.readthedocs.io/en/stable/api/pygama.vis.html#pygama.vis.waveform_browser.WaveformBrowser) is a pygama utility for accessing waveforms from raw files in an interactive way, enabling you to access, draw, or even process waveforms. Some use cases for this utility include investigating a population of waveforms, and debugging waveform processors.

Why do we need a waveform browser when we can access data via Pandas dataframes? Pandas dataframes work extremely well for reading tables of simple values from multiple HDF5 files. However, they are less optimal for waveforms. The reason for this is that they require holding all waveforms in memory at once. If we want to look at waveforms spread out across multiple files, this can potentially take up GBs of memory, which will cause problems! To get around this, we want to load only bits of the files into memory at a time and pull out only what we need. Since this is an inconvenient process, the `WaveformBrowser` will do this for you, while hiding the details as much as possible.

Let's start by importing necessary modules and test data:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, json

import pygama.lgdo.lh5_store as lh5
from pygama.vis.waveform_browser import WaveformBrowser
from legend_testdata import LegendTestData

ldata = LegendTestData()
raw_file = ldata.get_path("lh5/LDQTA_r117_20200110T105115Z_cal_geds_raw.lh5")

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 12

## Basic browsing

First, a minimal example simply drawing waveforms from the raw file. Let's create a minimal browser and draw the 50th waveform:

In [ ]:
browser = WaveformBrowser(raw_file, 'geds/raw')
browser.draw_entry(50)

To draw multiple waveforms in a single figure, provide a list if indices:

In [ ]:
browser.draw_entry([64, 82, 94])

Now draw the next waveform in the file. You can run this cell multiple times to scroll through many waveforms:

In [ ]:
browser.draw_next()

## Visualizing waveform transforms

Now, we'll shift from drawing populations of waveforms to drawing waveform transforms. We can draw any waveforms that are defined in a DSP JSON configuration file. This is useful for debugging purposes and for developing processors. We will draw the baseline subtracted waveform, pole-zero corrected waveform, and trapezoidal filter waveform. We will also draw horizontal and vertical lines for `trapE` (the maximum of the trapezoid) and `tp_0` (our estimate of the start of the waveform's rise). The browser will determine whether these lines should be horizontal or vertical based on the unit.

In [ ]:
browser = WaveformBrowser(
    raw_file, 'geds/raw',
    dsp_config="metadata/dsp-config.json", # Need to include a dsp config file!
    database={"pz_const": '180*us'},
    lines=['wf_blsub', 'wf_pz', 'wf_trap', 'trapEmax', 'tp_0'], # names of waveforms from dsp config file
    styles=[
        {'ls':['-'], 'c':['orange']},
        {'ls':[':'], 'c':['green']},
        {'ls':['--'], 'c':['blue']},
        {'lw':[0.5], 'c':['black']},
        {'lw':[0.5], 'c':['red']}],
    legend=['Waveform', 'PZ Corrected', "Trap Filter", 'Trap Max={trapEmax}', 't0={tp_0}'],
    legend_opts={'loc':"upper left"},
    x_lim=('35*us', '75*us') # x axis range
)

In [ ]:
browser.draw_next()